## 建造者模式

想象一下，我们想要一个由多个部分合成的对象，而且合成需要一步步的来操作。这个对象在其自身所有部分都完全创建之前是不完整的。这就是构造器模式能够帮助我们的地方。构造器模式通过自己的表现将一个复杂对象的构造进行分离。通过使用表现的构造分离，相同的构造介意被用到多个不同到表现上。

实际的例子可以帮助我们理解使用构造器的目的是什么。假设我们想要创建一个HTML页面生成器，基本的HTML页面结构都是相同的：以开始并以结束；内部HTML区域是和元素，内部头区域是<title>和</title>元素，等等。但是页面的表现又是不同的。每个页面都拥有自己的标题，自己的头，以及不同的内容。此外，页面通常的组成有多个步骤：一个函数添加标题，另一个添加主要头部，另外一个添加footer，等等。仅在整个页面合成完毕后，才使用最后的渲染函数将页面显示给用户。我们可以做到跟进一步，去扩展HTML生成器，这样它就可以生成完全不同的HTML页面了。一个页面或许包含表格、一个页面可能含有图片展示，另外一个页面还可能包含联系人表单，等等。

HTML页面生成器的问题可以使用构造器模式来解决。在这个模式中有两个主要的参与元素：构造器和经手人。构造器负责创建负责对象的多个部分。在HTML示例中，这些部分为主题，头部，主体，以及页面脚注。经手人使用构造器实例控制构建过程。HTML的例子中的部分为，标题，头部，主体，以及页面注脚。管理者使用一个构造器实例来控制构建过程。HTML的例子意味着要调用构造器的函数来设置标题，头部，等等。使用不同的构造器实例可以让我们不接触任何管理器的代码便可以创建不同的HTML页面。

构造器模式被用在了快餐店上。相同的步骤一直用在了准备汉堡和打包上（盒子、纸袋），即使存在不同类型的汉堡（原味，起司，等等）以及不同的包装（小号的盒子，中号的盒子，等等）原味汉堡和起司汉堡之间不同在于外观，而不是制造过程上。管理器是给全体工作人员发出需要准备什么食材的收银员，构造器是员工之中负责特殊订单的人。

### 使用案例
在我们知道对象必须用到多个步骤才能创建，以及需要相同构造的不同表现时才会使用构造器模式。这些需求存在于很多的应用中，比如页面生成器（类似于本章提到的页面生成器），文档转换器，以及用户界面（UI）的表单创建器。

在这一阶段，构造器模式和工厂模式之间的区别并不非常明显。主要的区别在于工厂模式以单步形式创建一个对象，而构造器模式则以踱步形式创建一个对象，并且几乎都是通过管理者的使用来实现的。一些以构造器模式为主要目标的实现，比如Java的StringBuilder 就忽略了管理者的使用，不过这是个例外。
另外一个区别是工厂模式立即返回创建的对象，而在构造器模式中用户代码显式地向经理人请求，在有需要时返回最终的对象。
新计算机的分析有助于区别构造器模式和工厂模式。假设你需要买一台新电脑。如果你决定买预先配置好的电脑，例如最新的Apple 1.4Ghz Mac mini，你使用的是工厂模式。所有硬件规格都已经由厂商预定义了，厂家不用问你也知道该干什么。厂商通常只会收到一条指令。
另外一个选择买一台定制的PC。在这个例子中，你可以使用构造器模式。你是给制造商（builder）下发个人定制电脑的经理人。

### Implementation 实现     
让我们来看一看，我们该如何使用构造器模式来编写一个披萨下单应用。披萨示例非常有趣，因为披萨要按照指定的订单一步一步地准备。要添加酱料的话，首先你需要准备生面团。要添加浇头，首先你需要放酱料。除非你把酱料和浇头都放到生面团上面否则你是没办法烘培披萨的。此外，通常，视面饼的厚度和浇头的使用，每个披萨都需要的不同的烘培时间。
我们从导入所需的模块开始，然后声明了几个Enum参数[j.mp/pytenum]，添加了在应用中多次应用的常量。如下， STEP_DELAY常量被用来在准备披萨（准备生面团，添加酱料，等等）的不同步骤中添加一个时间延迟：

In [2]:
from enum import Enum
import time

PizzaProgress = Enum('PizzaProgress', 'queued preparation baking ready')
PizzaDough = Enum('PizzaDough', 'thin thick')
PizzaSauce = Enum('PizzaSauce', 'tomato creme_fraiche')
PizzaTopping = Enum('PizzaTopping', 
                    'mozzarella double_mozzarella bacon ham mushrooms red_onion oregano')
STEP_DELAY = 3 # in seconds for the sake of the example

我们最终的产品是‘披萨’，它由Pizza类描述。在使用构造器模式时，终端产品并没有太多的责任，因为它不支持直接实例化。构造器创建终端产品的实例，以保证披萨正确备料了。这就是为什么Pizza类这么细小的原因。基本上这个类初始化所有数据，以唤醒默认值。处理异常的方法是prepare_dough()。prepare_dough()方法定义在了Pizza类而不是构造器中有两个理由：


要说明的是，终端产品通常是最小化的，但是这并不意味着你从此就不能让它承担任何的职责

通过使用合成来提升代码的复用性

In [7]:
class Pizza:
    def __init__(self, name):
        self.name = name
        self.dough = None
        self.sauce = None
        self.topping = []

    def __str__(self):
        return self.name

    def prepare_dough(self, dough):
        self.dough = dough
        print('preparing the {} dough of your {}...'.format(self.dough.name,self))
        time.sleep(STEP_DELAY)
        print('done with the {} dough'.format(self.dough.name))

有两个构造器：一个用来创建玛格丽塔披萨（MargaritaBuilder），另外一个用来创建奶油培根披萨（CreamyBaconBuilder）。每个构造器创建一个Pizza实例，以及包含了遵循披萨制作流程的方法：prepare_dough(), add_sauce(), add_topping(), and bake()。准确地来说，prepare_dough()只是一个应用于Pizza类的方法*prepare_dough()*的包装器。请注意每个构造器是如何处理定制披萨的细节。例如，马格丽特披萨的浇头是‘双份白干奶酪’和‘牛至叶’，而奶油培根披萨的浇头则是‘白干奶酪’，‘培根’，‘火腿’，‘蘑菇’，‘紫皮洋葱’，和‘牛至叶’，一如下面所示：

In [9]:
class MargaritaBuilder:
    def __init__(self):
        self.pizza = Pizza('margarita')
        self.progress = PizzaProgress.queued
        self.baking_time = 5 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thin)

    def add_sauce(self):
        print('adding the tomato sauce to your margarita...')
        self.pizza.sauce = PizzaSauce.tomato
        time.sleep(STEP_DELAY)
        print('done with the tomato sauce')

    def add_topping(self):
        topping_desc = 'double mozzarella, oregano'
        topping_items = (PizzaTopping.double_mozzarella, PizzaTopping.oregano)
        print('adding the topping ({}) to your margarita'.format(topping_desc))
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print('done with the topping ({})'.format(topping_desc))

    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your margarita for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your margarita is ready')

        
class CreamyBaconBuilder:
    def __init__(self):
        self.pizza = Pizza('creamy bacon')
        self.progress = PizzaProgress.queued
        self.baking_time = 7 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thick)

    def add_sauce(self):
        print('adding the crème fraîche sauce to your creamy bacon')
        self.pizza.sauce = PizzaSauce.creme_fraiche
        time.sleep(STEP_DELAY)
        print('done with the crème fraîche sauce')

    def add_topping(self):
        topping_desc = 'mozzarella, bacon, ham, mushrooms, red onion, oregano'
        topping_items =  (PizzaTopping.mozzarella,
                          PizzaTopping.bacon,
                          PizzaTopping.ham,
                          PizzaTopping.mushrooms,
                          PizzaTopping.red_onion, 
                          PizzaTopping.oregano)
        print('adding the topping ({}) to your creamy bacon'.format(topping_desc))
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print('done with the topping ({})'.format(topping_desc))

    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your creamy bacon for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your creamy bacon is ready')


这个例子中的管理器是服务员。Waiter类的核心是*construct_pizza()*方法，这个方法接受一个构造器并作为参数，然后按照正确的顺序执行所有的披萨预备工作。选择合适的构造器，甚至可以在运行时实现，构造器给予我们能够不用修改管理器（Waiter）的任何代码就可以常见不同风味的披萨。

In [10]:
class Waiter:
    def __init__(self):
        self.builder = None

    def construct_pizza(self, builder):
        self.builder = builder
        steps = (builder.prepare_dough, 
                 builder.add_sauce, 
                 builder.add_topping, 
                 builder.bake)
        [step() for step in steps]

    @property
    def pizza(self):
        return self.builder.pizza


validate_style()函数类似于在工厂模式中描述过的validate_age()函数。它用来保证用户给出有效的输入，而在这个例子它是一个映射到披萨供应商的字符。字符m使用的MargaritaBuilder类，字符c使用CreamyBaconBuilder类。

In [11]:
def validate_style(builders):
    try:
        input_msg = 'What pizza would you like, [m]argarita or [c]reamy bacon? '
        pizza_style = input(input_msg)
        builder = builders[pizza_style]()
        valid_input = True
    except KeyError:
        error_msg = 'Sorry, only margarita (key m) and creamy bacon (key c) are available'
        print(error_msg)
        return (False, None)
    return (True, builder)

最后部分是*main()*函数。main()函数包含了初始化披萨构造器的代码。披萨构造器之后被Waiter经理人用来预备披萨。创建的披萨可以在之后的任意时刻送达用户：

In [8]:
from enum import Enum
import time

PizzaProgress = Enum('PizzaProgress', 'queued preparation baking ready')
PizzaDough = Enum('PizzaDough', 'thin thick')
PizzaSauce = Enum('PizzaSauce', 'tomato creme_fraiche')
PizzaTopping = Enum('PizzaTopping', 
                    'mozzarella double_mozzarella bacon ham mushrooms red_onion oregano')
STEP_DELAY = 3 # in seconds for the sake of the example


class Pizza:
    def __init__(self, name):
        self.name = name
        self.dough = None
        self.sauce = None
        self.topping = []

    def __str__(self):
        return self.name

    def prepare_dough(self, dough):
        self.dough = dough
        print('preparing the {} dough of your {}...'.format(self.dough.name,self))
        time.sleep(STEP_DELAY)
        print('done with the {} dough'.format(self.dough.name))

        
class MargaritaBuilder:
    def __init__(self):
        self.pizza = Pizza('margarita')
        self.progress = PizzaProgress.queued
        self.baking_time = 5 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thin)

    def add_sauce(self):
        print('adding the tomato sauce to your margarita...')
        self.pizza.sauce = PizzaSauce.tomato
        time.sleep(STEP_DELAY)
        print('done with the tomato sauce')

    def add_topping(self):
        topping_desc = 'double mozzarella, oregano'
        topping_items = (PizzaTopping.double_mozzarella, PizzaTopping.oregano)
        print('adding the topping ({}) to your margarita'.format(topping_desc))
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print('done with the topping ({})'.format(topping_desc))

    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your margarita for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your margarita is ready')

        
class CreamyBaconBuilder:
    def __init__(self):
        self.pizza = Pizza('creamy bacon')
        self.progress = PizzaProgress.queued
        self.baking_time = 7 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thick)

    def add_sauce(self):
        print('adding the crème fraîche sauce to your creamy bacon')
        self.pizza.sauce = PizzaSauce.creme_fraiche
        time.sleep(STEP_DELAY)
        print('done with the crème fraîche sauce')

    def add_topping(self):
        topping_desc = 'mozzarella, bacon, ham, mushrooms, red onion, oregano'
        topping_items =  (PizzaTopping.mozzarella,
                          PizzaTopping.bacon,
                          PizzaTopping.ham,
                          PizzaTopping.mushrooms,
                          PizzaTopping.red_onion, 
                          PizzaTopping.oregano)
        print('adding the topping ({}) to your creamy bacon'.format(topping_desc))
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print('done with the topping ({})'.format(topping_desc))

    def bake(self):
        self.progress = PizzaProgress.baking
        print('baking your creamy bacon for {} seconds'.format(self.baking_time))
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your creamy bacon is ready')

        
class Waiter:
    def __init__(self):
        self.builder = None

    def construct_pizza(self, builder):
        self.builder = builder
        steps = (builder.prepare_dough, 
                 builder.add_sauce, 
                 builder.add_topping, 
                 builder.bake)
        [step() for step in steps]

    @property
    def pizza(self):
        return self.builder.pizza

        
def validate_style(builders):
    try:
        input_msg = 'What pizza would you like, [m]argarita or [c]reamy bacon? '
        pizza_style = input(input_msg)
        builder = builders[pizza_style]()
        valid_input = True
    except KeyError:
        error_msg = 'Sorry, only margarita (key m) and creamy bacon (key c) are available'
        print(error_msg)
        return (False, None)
    return (True, builder)

    
def main():
    builders = dict(m=MargaritaBuilder, c=CreamyBaconBuilder)
    valid_input = False
    while not valid_input:
        valid_input, builder = validate_style(builders)
    print()
    waiter = Waiter()
    waiter.construct_pizza(builder)
    pizza = waiter.pizza
    print()
    print('Enjoy your {}!'.format(pizza))

In [6]:
main()

What pizza would you like, [m]argarita or [c]reamy bacon? m

preparing the thin dough of your margarita...
done with the thin dough
adding the tomato sauce to your margarita...
done with the tomato sauce
adding the topping (double mozzarella, oregano) to your margarita
done with the topping (double mozzarella, oregano)
baking your margarita for 5 seconds
your margarita is ready

Enjoy your margarita!


### 与软件相关的例子
The HTML example that was mentioned at the beginning of the chapter is actually used by django-widgy, a third-party tree editor for Django that can be used as a Content Management System (CMS). The django-widgy editor contains a page builder that can be used for creating HTML pages with different layouts [j.mp/widgypb].

本章开始提到的HTML例子实际上是使用的是django-widgy，这是一个第三方的Django树形编辑器包，它能够当作CMS来使用。django-widgy包含了一个构造器，它能够用来创建不同布局的HTML页面。

The django-query-builder library is another third-party Django library that relies on the Builder pattern. The django-query-builder library can be used for building SQL queries dynamically. Using this, we can control all aspects of a query and create a different range of queries, from simple to very complex [j.mp/djangowidgy].

django-query-builder库是另外一个第三方的Django库，它依赖于构造器模式。django-query-builder库用来构建动态SQL查询。使用这个库，我们可以控制查询和创建一组不同查询的方方面面，从简单的到复杂都可以。